In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json

In [2]:
url = 'http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/leis-ordinarias/2020-leis-ordinarias'
url2 = 'http://www.planalto.gov.br/ccivil_03/_ato2019-2022/2020/lei/L14013.htm'
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url2)
time.sleep(10)
#--pegando o código fonte inteiro da página --#
elements = driver.page_source
print(elements)
driver.quit()

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml"><head>
  <meta content="Microsoft FrontPage 6.0" name="GENERATOR">
  <title>
   L14013
  </title>
  <style>
   <!--
 table.MsoNormalTable
	{mso-style-parent:"";
	font-size:10.0pt;
	font-family:"Times New Roman"
	}
h1
	{margin-bottom:.0001pt;
	text-align:justify;
	line-height:12.0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal; margin-left:0pt; margin-right:0pt; margin-top:0pt}
h2
	{margin-top:12.0pt;
	margin-right:0pt;
	margin-bottom:3.0pt;
	margin-left:0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal}
span.Hiperlink
	{mso-style-parent:"";
	color:blue;
	text-decoration:underline;
	text-underline:single}
h6
	{margin-bottom:.0001pt;
	text-align:center;
	line-height:150%;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:Arial;
	margin-left:0cm; 

In [3]:
#2. Parsear o conteúdo
soup = BeautifulSoup(elements, 'lxml')

In [4]:
#myp = soup.findAll('p', class_='Artigo') neste caso a class é outra
myp = soup.findAll('p', class_='texto1')
myp

[<p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style="font-size: 10.0pt; font-family: Arial,sans-serif; color: black">
 <a name="art1"></a>Art. 1</span><span style='font-size:10.0pt;
 font-family:"Arial","sans-serif";color:black'>º  No mês de 
 	janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove 
 	reais).</span></p>,
 <p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style='font-size:10.0pt;font-family:"Arial","sans-serif";color:black'>
 	Parágrafo único. Em decorrência do disposto no <b>caput</b> deste artigo, o 
 	valor diário do salário-mínimo corresponderá, no mês de janeiro de 2020, a 
 	R$ 34,63 (trinta e quatro reais e sessenta e três centavos) e o valor 
 	horário, a R$ 4,72 (quatro reais e setenta e dois centavos).</span></p>,
 <p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style=

In [16]:
lista = []
for parag in soup.find_all('p', class_='texto1'):
    for sp in parag.find_all('span'):
        print(sp.text)
        lista.append(sp.text)


Art. 1
º  No mês de 
	janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove 
	reais).

	Parágrafo único. Em decorrência do disposto no caput deste artigo, o 
	valor diário do salário-mínimo corresponderá, no mês de janeiro de 2020, a 
	R$ 34,63 (trinta e quatro reais e sessenta e três centavos) e o valor 
	horário, a R$ 4,72 (quatro reais e setenta e dois centavos).



	Art. 2º A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 
	1.045,00 (mil e quarenta e cinco reais).

	Parágrafo único. Em decorrência do disposto no caput deste artigo, o 
	valor diário do salário-mínimo corresponderá, a partir de 1º de fevereiro de 
	2020, a R$ 34,83 (trinta e quatro reais e oitenta e três centavos) e o valor 
	horário, a R$ 4,75 (quatro reais e setenta e cinco centavos).



	Art. 3º Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.



	Art. 4º Esta Lei entra em vigor na data de sua publicação. 


In [30]:
#removendo caractere indesejável
for i in range(len(lista)):
    lista[i] = lista[i].replace('\n\t', '')
    lista[i] = lista[i].replace('\n', '')
    lista[i] = lista[i].replace('\xa0', '')
lista[0]

'Art. 1º No mês de janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove reais).'

In [19]:
#consertando o item 0
lista[0] = (str(lista[0]) + str(lista[1]))
lista[0]

'Art. 1º No mês de janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove reais).'

In [20]:
#preparando as listas para formar um dataframe
lista_artigos = []
lista_colunas = []
for artigo in lista:
    if 'Art.' in artigo:
        lista_colunas.append(artigo[0:7])
        lista_artigos.append(artigo[7:])
print(lista_artigos)
print(lista_colunas)

[' No mês de janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove reais).', ' A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 1.045,00 (mil e quarenta e cinco reais).', ' Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.', ' Esta Lei entra em vigor na data de sua publicação.']
['Art. 1º', 'Art. 2º', 'Art. 3º', 'Art. 4º']


In [22]:
print(lista_artigos)
print(lista_colunas)

[' No mês de janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove reais).', ' A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 1.045,00 (mil e quarenta e cinco reais).', ' Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.', ' Esta Lei entra em vigor na data de sua publicação.']
['Art. 1º', 'Art. 2º', 'Art. 3º', 'Art. 4º']


In [27]:
for i in range(len(lista_artigos)):
    print(len(lista_artigos[i]))

93
106
69
51


In [28]:
#criando o dataframe
df = pd.DataFrame(lista_artigos).T
df.columns = lista_colunas

In [29]:
df

,Art. 1º,Art. 2º,Art. 3º,Art. 4º
0,"No mês de janeiro de 2020, o salário-mínimo s...","A partir de 1º de fevereiro de 2020, o salári...","Fica revogada a Medida Provisória nº 916, de ...",Esta Lei entra em vigor na data de sua public...


# Carregar outra página que contenha incisos

In [3]:
url = 'http://www.planalto.gov.br/ccivil_03/_ato2019-2022/2020/lei/L14002.htm'
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url)
time.sleep(10)
#--pegando o código fonte inteiro da página --#
elements = driver.page_source
print(elements)
driver.quit()

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml"><head>
  <meta content="Microsoft FrontPage 6.0" name="GENERATOR">
  <title>
   L14002
  </title>
  <style>
   <!--
 table.MsoNormalTable
	{mso-style-parent:"";
	font-size:10.0pt;
	font-family:"Times New Roman"
	}
h1
	{margin-bottom:.0001pt;
	text-align:justify;
	line-height:12.0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal; margin-left:0pt; margin-right:0pt; margin-top:0pt}
h2
	{margin-top:12.0pt;
	margin-right:0pt;
	margin-bottom:3.0pt;
	margin-left:0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal}
span.Hiperlink
	{mso-style-parent:"";
	color:blue;
	text-decoration:underline;
	text-underline:single}
h6
	{margin-bottom:.0001pt;
	text-align:center;
	line-height:150%;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:Arial;
	margin-left:0cm; 

In [5]:
#2. Parsear o conteúdo
soup = BeautifulSoup(elements, 'lxml')

In [6]:
myp = soup.findAll('p', class_='texto1')
myp

[<p align="center" class="texto1" style="text-align: center; text-indent: 1.0cm; line-height: normal; margin-top: 20px; margin-bottom: 20px">
 <span style='font-size:10.0pt;font-family:"Arial","sans-serif"'>
 	CAPÍTULO I</span></p>,
 <p align="center" class="texto1" style="text-align: center; text-indent: 1.0cm; line-height: normal; margin-top: 20px; margin-bottom: 20px">
 <span style='font-size:10.0pt;font-family:"Arial","sans-serif"'>
 	DA PRORROGAÇÃO DE BENEFÍCIOS TRIBUTÁRIOS</span></p>,
 <p align="justify" class="texto1" style="text-indent: 1.0cm; line-height: normal; margin-top: 20px; margin-bottom: 20px">
 <span style="font-size: 10.0pt; font-family: Arial,sans-serif">
 <a name="art1"></a>Art. 1</span><span style='font-size:10.0pt;font-family:"Arial","sans-serif"'>º 
 	 O art. 16 da <a href="../../../_Ato2004-2006/2006/Lei/L11371.htm">Lei nº 11.371, de 28 de novembro de 2006</a>, passa a vigorar com 
 	a seguinte redação:         </span>
 <font face="Arial" size="2">
 <span style

In [8]:
lista = []
for parag in soup.find_all('p', class_='texto1'):
    for sp in parag.find_all('span'):
        print(sp.text)
        lista.append(sp.text)


	CAPÍTULO I

	DA PRORROGAÇÃO DE BENEFÍCIOS TRIBUTÁRIOS

Art. 1
º 
	 O art. 16 da Lei nº 11.371, de 28 de novembro de 2006, passa a vigorar com 
	a seguinte redação:         

(Produção de efeito) 


“Art. 16.  Em relação aos fatos geradores que ocorrerem até 31 de 
			dezembro de 2022, a alíquota do imposto sobre a renda retido na 
			fonte incidente nas operações de que trata o inciso V do caput 
			do art. 1º da Lei nº 9.481, de 13 de agosto de 1997, na hipótese de 
			pagamento, crédito, entrega, emprego ou remessa, por fonte situada 
			no País, a pessoa jurídica domiciliada no exterior, a título de 
			contraprestação de contrato de arrendamento mercantil de aeronave ou 
			de motores destinados a aeronaves, celebrado por empresa de 
			transporte aéreo público regular, de passageiros ou cargas, 
			corresponderá a:

			I – (VETADO);

			II – 1,5% (um inteiro e cinco décimos por cento), entre 1º de 
			janeiro de 2020 e 31 de dezembro de 2020.” (NR)



	Art. 2º  (VETADO).       


In [18]:
#removendo caractere indesejável
for i in range(len(lista)):
    lista[i] = lista[i].replace('\n\t', '')
    lista[i] = lista[i].replace('\n', '')
    lista[i] = lista[i].replace('\xa0', '')
    lista[i] = lista[i].replace('\t\t', '')
lista[0:2]

['CAPÍTULO I', 'DA PRORROGAÇÃO DE BENEFÍCIOS TRIBUTÁRIOS']

In [19]:
#começa a partir daqui:
lista = lista[2:]

In [23]:
lista[0]

'Art. 1'

In [24]:
#consertando o item 0
lista[0] = (str(lista[0]) + str(lista[1]))
lista[0]

'Art. 1º O art. 16 da Lei nº 11.371, de 28 de novembro de 2006, passa a vigorar com a seguinte redação: '

In [26]:
#preparando as listas para formar um dataframe
lista_artigos = []
lista_colunas = []
for artigo in lista:
    if 'Art.' in artigo:
        lista_colunas.append(artigo[0:7])
        lista_artigos.append(artigo[7:])

In [31]:
print(len(lista_artigos))
print(len(lista_colunas))

47
47


In [40]:
#limpando lista_colunas:
"""for i in range(len(lista_colunas)):
    lista_colunas[i] = lista_colunas[i].replace('“', '')"""
#acrescentando o º ao final
for i in range(len(lista_colunas)):
    if lista_colunas[i][-1] != 'º':
        lista_colunas[i] += 'º'
lista_colunas

['Art. 1º',
 'Art. 1º',
 'Art. 2º',
 'Art. 3º',
 'Art. 4º',
 'Art. 5º',
 'Art. 6º',
 'Art. 7º',
 'Art. 8º',
 'Art. 9º',
 'Art. 10º',
 'Art. 11º',
 'Art. 12º',
 'Art. 13º',
 'Art. 14º',
 'Art. 15º',
 'Art. 16º',
 'Art. 17º',
 'Art. 18º',
 'Art. 19º',
 'Art. 20º',
 'Art. 21º',
 'Art. 22º',
 'Art. 23º',
 'Art. 24º',
 'Art. 25º',
 'Art. 26º',
 'Art. 27º',
 'Art. 28º',
 'Art. 29º',
 'Art. 30º',
 'Art. 31º',
 'Art. 32º',
 'Art. 8º',
 'Art. 8º',
 'Art. 8º',
 'Art. 8º',
 'Art. 8º',
 'Art. 8º',
 'Art. 8º',
 'Art. 1º',
 'Art. 33º',
 'Art. 6º',
 'Art. 34º',
 'Art. 35º',
 'Art. 36º',
 'Art. 37º']

In [41]:
#criando o dataframe
df = pd.DataFrame(lista_artigos).T
df.columns = lista_colunas

In [42]:
df

,Art. 1º,Art. 1º,Art. 2º,Art. 3º,Art. 4º,Art. 5º,Art. 6º,Art. 7º,Art. 8º,Art. 9º,...,Art. 8º,Art. 8º,Art. 8º,Art. 1º,Art. 33º,Art. 6º,Art. 34º,Art. 35º,Art. 36º,Art. 37º
0,"O art. 16 da Lei nº 11.371, de 28 de novembro...",6. Em relação aos fatos geradores que ocorrere...,(VETADO).,Fica o Poder Executivo federal autorizado a i...,Compete à Embratur:,Fica a Embratur autorizada a:,São órgãos de direção da Embratur:,O Conselho Deliberativo será composto:,O Conselho Fiscal será composto de 2 (dois) r...,A Diretoria-Executiva da Embratur será compos...,...,º-M. A avaliação institucional considerada par...,º-N. A avaliação individual do servidor será r...,"º-O. O órgão, a entidade ou a organização de e...",2. É instituída a Gratificação de Qualificação...,". O art. 63 da Lei nº 12.462, de 4 de agosto d...",3. ..............................................,". Em caso de guerra, convulsão social, calamid...",. Ato do Poder Executivo federal regulamentará...,. Revogam-se:,. Esta Lei entra em vigor na data de sua publi...
